In [1]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt

from transToFeature import trans2feature

In [2]:
ori_path = "/home/battery/battery experiment/jsonData/"
B0005_discharge_path = ori_path + "B0005_discharge.json"
B0006_discharge_path = ori_path + "B0006_discharge.json"
B0007_discharge_path = ori_path + "B0007_discharge.json"
B0018_discharge_path = ori_path + "B0018_discharge.json"

discharge_data05 = json.load(open(B0005_discharge_path))
discharge_data06 = json.load(open(B0006_discharge_path))
discharge_data07 = json.load(open(B0007_discharge_path))
discharge_data18 = json.load(open(B0018_discharge_path))

D05 = trans2feature(discharge_data05)
D06 = trans2feature(discharge_data06)
D07 = trans2feature(discharge_data07)
D18 = trans2feature(discharge_data18)

In [3]:
features = pd.concat([D05, D06, D07, D18])
features.head()

,period,avg_voltage,terminate_voltage,min_temp,max_temp,avg_temp,avg_voltage_load,terminate_voltage_load,begin_capacity,capacity
0,0,3.529829,2.612467,24.325993,38.982181,32.572328,2.404944,1.782,1.856487,1.856487
1,1,3.537320,2.587209,24.685948,39.033398,32.725235,2.399260,1.766,1.856487,1.846327
2,2,3.543737,2.651917,24.734266,38.818797,32.642862,2.397969,1.820,1.856487,1.835349
3,3,3.543666,2.592948,24.652244,38.762305,32.514876,2.408289,1.772,1.856487,1.835263
4,4,3.542343,2.547420,24.518700,38.665393,32.382349,2.408505,1.738,1.856487,1.834646


In [33]:
features.drop(['period'],inplace=True,axis=1)
features.head()

,avg_voltage,terminate_voltage,min_temp,max_temp,avg_temp,avg_voltage_load,terminate_voltage_load,begin_capacity,capacity
0,3.529829,2.612467,24.325993,38.982181,32.572328,2.404944,1.782,1.856487,1.856487
1,3.537320,2.587209,24.685948,39.033398,32.725235,2.399260,1.766,1.856487,1.846327
2,3.543737,2.651917,24.734266,38.818797,32.642862,2.397969,1.820,1.856487,1.835349
3,3.543666,2.592948,24.652244,38.762305,32.514876,2.408289,1.772,1.856487,1.835263
4,3.542343,2.547420,24.518700,38.665393,32.382349,2.408505,1.738,1.856487,1.834646


# LGB

In [4]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import mean_squared_error

In [5]:
label = features["capacity"]
fea = features.drop(labels = ["capacity"], axis = 1)

x_train, x_test, y_train, y_test = train_test_split(fea, label, test_size = 0.2, random_state = 5)

In [6]:
lgb_train = lgb.Dataset(x_train,y_train)
lgb_eval = lgb.Dataset(x_test,y_test,reference=lgb_train)

In [7]:
x_test.shape

(128, 9)

In [8]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression',
    'metric': {'l2', 'l1'},
    'num_leaves': 15,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

print('Starting training...')
# train
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=500,
                valid_sets=lgb_eval,
                early_stopping_rounds=10)

print('Saving model...')
# save model to file
gbm.save_model('model.txt')

print('Starting predicting...')
# predict
y_pred = gbm.predict(x_test, num_iteration=gbm.best_iteration)
# eval
print('The rmse of prediction is:', mean_squared_error(y_test, y_pred) ** 0.5)

Starting training...
[1]	valid_0's l2: 0.0394267	valid_0's l1: 0.174378
Training until validation scores don't improve for 10 rounds.
[2]	valid_0's l2: 0.0358309	valid_0's l1: 0.166121
[3]	valid_0's l2: 0.0324882	valid_0's l1: 0.158067
[4]	valid_0's l2: 0.0295115	valid_0's l1: 0.150694
[5]	valid_0's l2: 0.0268318	valid_0's l1: 0.143589
[6]	valid_0's l2: 0.0244424	valid_0's l1: 0.13691
[7]	valid_0's l2: 0.0222975	valid_0's l1: 0.130666
[8]	valid_0's l2: 0.0203506	valid_0's l1: 0.124759
[9]	valid_0's l2: 0.0186157	valid_0's l1: 0.119314
[10]	valid_0's l2: 0.0169373	valid_0's l1: 0.113615
[11]	valid_0's l2: 0.0153698	valid_0's l1: 0.108096
[12]	valid_0's l2: 0.0139596	valid_0's l1: 0.102902
[13]	valid_0's l2: 0.0127026	valid_0's l1: 0.0981117
[14]	valid_0's l2: 0.0115969	valid_0's l1: 0.0937411
[15]	valid_0's l2: 0.0105703	valid_0's l1: 0.089436
[16]	valid_0's l2: 0.0096306	valid_0's l1: 0.0852767
[17]	valid_0's l2: 0.00879343	valid_0's l1: 0.0814056
[18]	valid_0's l2: 0.00802529	valid_0'

[150]	valid_0's l2: 0.000167854	valid_0's l1: 0.010196
[151]	valid_0's l2: 0.0001676	valid_0's l1: 0.0101982
[152]	valid_0's l2: 0.000167228	valid_0's l1: 0.0101919
[153]	valid_0's l2: 0.000166851	valid_0's l1: 0.0101859
[154]	valid_0's l2: 0.00016652	valid_0's l1: 0.0101799
[155]	valid_0's l2: 0.000165489	valid_0's l1: 0.010142
[156]	valid_0's l2: 0.000164948	valid_0's l1: 0.0101282
[157]	valid_0's l2: 0.000164857	valid_0's l1: 0.0101209
[158]	valid_0's l2: 0.000163916	valid_0's l1: 0.0100859
[159]	valid_0's l2: 0.000163062	valid_0's l1: 0.0100622
[160]	valid_0's l2: 0.000162686	valid_0's l1: 0.0100446
[161]	valid_0's l2: 0.000162709	valid_0's l1: 0.0100429
[162]	valid_0's l2: 0.00016336	valid_0's l1: 0.0100583
[163]	valid_0's l2: 0.000163867	valid_0's l1: 0.010079
[164]	valid_0's l2: 0.000164029	valid_0's l1: 0.0100819
[165]	valid_0's l2: 0.000164853	valid_0's l1: 0.0101109
[166]	valid_0's l2: 0.000164094	valid_0's l1: 0.0100893
[167]	valid_0's l2: 0.000163901	valid_0's l1: 0.0100844

In [9]:
print('The rmse of prediction is:', mean_squared_error(y_test, y_pred) ** 0.5)

The rmse of prediction is: 0.011820154119821872


# FFM

In [1]:
import xlearn as xl
import pandas as pd

In [10]:
x_data = pd.concat([y_train,x_train],axis=1)
y_data = pd.concat([y_test,x_test,],axis=1)

x_data.to_csv('data/train1.txt',index=False,sep='\t')
y_data.to_csv('data/test1.txt',index=False,sep='\t')

In [26]:
path='data/'
ffm_model = xl.create_ffm()
ffm_model.setTrain(path+'train1.txt')
ffm_model.setTest(path+'test1.txt')
param = {'task':'reg', 'lr':0.01, 'lambda':0.001,'metric': 'rmse','opt':'ftrl','epoch':200,'k':4,
         'alpha': 1.5, 'beta': 0.01, 'lambda_1': 0.0, 'lambda_2': 0.0}
ffm_model.fit(param,"./model.out")
ffm_model.predict("./model.out","./output.txt")

In [27]:
y_data = pd.read_csv('data/test1.csv')
y_data.head()

,period,avg_voltage,terminate_voltage,min_temp,max_temp,avg_temp,avg_voltage_load,terminate_voltage_load,begin_capacity,capacity
0,71,3.493996,2.472084,23.372100,37.919194,31.228216,2.373095,1.631,1.855005,1.523845
1,144,3.468682,1.994727,23.758944,41.024915,32.581732,2.392769,1.116,1.891052,1.446757
2,140,3.473377,2.632271,24.071744,41.064605,33.411988,2.132664,1.718,1.856487,1.344189
3,85,3.504277,2.028778,23.380469,40.634569,32.060652,2.481172,1.146,1.891052,1.595917
4,26,3.536321,2.490470,23.958513,37.598248,31.415703,2.487640,1.657,1.855005,1.722231


In [28]:
y_test = y_data['capacity']
y_pre_ffm = pd.read_csv('./output.txt',header=None)
y_pre_ffm.shape,y_test.shape

((128, 1), (128,))

In [29]:
print('The rmse of prediction is:', mean_squared_error(y_test, y_pre_ffm) ** 0.5)

The rmse of prediction is: 0.08988908188364432


# LR

In [ ]:
lr_model = xl.create_linear()

In [36]:
from sklearn.linear_model import LinearRegression

In [37]:
LR = LinearRegression()
LR.fit(x_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [38]:
y_pred = LR.predict(x_test)
print("MSE:",metrics.mean_squared_error(y_test, y_pred))
print("RMSE:",np.sqrt(metrics.mean_squared_error(y_test, y_pred)))

MSE: 0.0008553121193846222
RMSE: 0.029245719676298312
